### Upload the File

In [ ]:
!rm -rf hf_sequence_classifier.py authors_dataset.py


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving austen_utf8.txt to austen_utf8 (9).txt
Saving authors_dataset.py to authors_dataset.py
Saving data_processor.py to data_processor (10).py
Saving dickens_utf8.txt to dickens_utf8 (9).txt
Saving hf_sequence_classifier.py to hf_sequence_classifier.py
Saving tolstoy_utf8.txt to tolstoy_utf8 (9).txt
Saving wilde_utf8.txt to wilde_utf8 (9).txt


In this Notebook I will be testing the sequence classifier class

In [ ]:
from data_processor import DataProcessor
import hf_sequence_classifier as hf
import os
# from dotenv import load_dotenv

### Load Data

In [ ]:
# load_dotenv()
# data_dir = os.getenv('TRAIN_DATA_PATH')

In [ ]:
data_dir = "/content"

author_files = ["austen_utf8.txt", "dickens_utf8.txt", "tolstoy_utf8.txt", "wilde_utf8.txt"]
author_names = ["Austen", "Dickens", "Tolstoy", "Wilde"]

data_proc = DataProcessor()

authors_train_data = {}
authors_test_data = {}

for author_name, author_file in zip(author_names, author_files):
    print(f"Processing data for author: {author_name}")
    file_path = os.path.join(data_dir, author_file)

    if os.path.exists(file_path):
        trainset, devset = data_proc.process_split_file(file_path)
        authors_train_data[author_name] = trainset
        authors_test_data[author_name] = devset
    else:
        print(f"Warning: File {file_path} not found!")

Processing data for author: Austen
Splitting into training and development...
Processing data for author: Dickens
Splitting into training and development...
Processing data for author: Tolstoy
Splitting into training and development...
Processing data for author: Wilde
Splitting into training and development...


Sampling since the whole dataset took too much

In [ ]:
import random

authors_train_data_sample = {}

# Sample 50 entries per author
for author_name, trainset in authors_train_data.items():
    authors_train_data_sample[author_name] = random.sample(trainset, min(50, len(trainset)))


In [ ]:
import importlib
importlib.reload(hf)

<module 'hf_sequence_classifier' from '/content/hf_sequence_classifier.py'>

### Training the Model

In [ ]:
model = hf.SequenceClassifier(num_labels=4,model_name="distilbert-base-uncased")

In [ ]:
model.train(authors_train_data_sample)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,1.396600
20,1.385000
30,1.392900


Model and tokenizer saved to ./results


### Testing the Model on the Devset

### Download the weights of the model

In [ ]:
import shutil

shutil.make_archive("/content/results_backup", 'zip', "./results")


'/content/results_backup.zip'

In [ ]:
from google.colab import files

files.download("/content/results_backup.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import time

In [24]:
print("id2label mapping:", model.id2label)


id2label mapping: {0: 'Austen', 1: 'Dickens', 2: 'Tolstoy', 3: 'Wilde'}


In [26]:
print("Expected num_labels:", len(authors_train_data_sample.keys()))
print("Model num_labels:", model.num_labels)

Expected num_labels: 4
Model num_labels: 4


In [27]:
print("label2id mapping :",model.label2id)

label2id mapping : {'Austen': 0, 'Dickens': 1, 'Tolstoy': 2, 'Wilde': 3}


In [23]:
start_time = time.time()
pred = model.evaluate_devset(authors_test_data,show_accuracy=True)
end_time = time.time()
execution_time = end_time - start_time
print(f"Training completed in {execution_time:.2f} seconds.")

Results on dev set:


KeyError: 52